In [6]:
import psycopg2
import os

In [7]:
# Informações de conexão com o PostgreSQL
DB_HOST = "airflow-postgres"
DB_PORT = "5432"
DB_USER = os.getenv('POSTGRES_USER')
DB_PASSWORD = os.getenv('POSTGRES_PASSWORD')
DB_NAME = "azurecost"

conn = None

In [8]:
try:
    # Conecta-se ao banco de dados 'postgres' para criar o banco 'azurecost'
    conn = psycopg2.connect(
        host=DB_HOST,
        port=DB_PORT,
        database="postgres",
        user=DB_USER,
        password=DB_PASSWORD
    )
    conn.autocommit = True
    cur = conn.cursor()

    # Verifica e cria o banco de dados 'azurecost' se não existir
    cur.execute(f"SELECT 1 FROM pg_database WHERE datname='{DB_NAME}'")
    exists = cur.fetchone()
    if not exists:
        cur.execute(f"CREATE DATABASE {DB_NAME}")
        print(f"Banco de dados '{DB_NAME}' criado com sucesso.")
    else:
        print(f"Banco de dados '{DB_NAME}' já existe.")

    cur.close()
    conn.close()  # Fecha a conexão com 'postgres'

    # Conecta-se ao banco de dados 'azurecost' para criar a tabela
    conn = psycopg2.connect(
        host=DB_HOST,
        port=DB_PORT,
        database=DB_NAME,
        user=DB_USER,
        password=DB_PASSWORD
    )
    conn.autocommit = True
    cur = conn.cursor()

    # Comando SQL para criar a tabela 'azure_cost_data'
    create_table_query = """
    CREATE TABLE IF NOT EXISTS azure_cost_data (
        ResourceName VARCHAR,
        SubscriptionId VARCHAR,
        ResourceGroup VARCHAR,
        Provider VARCHAR,
        StatusRecourse VARCHAR,
        PreTaxCost DOUBLE PRECISION,
        Pct_Change DOUBLE PRECISION,
        Currency VARCHAR,
        UsageDate DATE,
        TendenciaCusto VARCHAR,
        PrevisaoProxima DOUBLE PRECISION
    );
    """

    cur.execute(create_table_query)
    conn.commit()
    print(f"Tabela 'azure_cost_data' criada com sucesso no banco de dados '{DB_NAME}'.")

    cur.close()

except psycopg2.Error as e:
    print(f"Erro ao conectar ou criar o banco de dados/tabela: {e}")
finally:
    if conn:
        conn.close()

Banco de dados 'azurecost' já existe.
Tabela 'azure_cost_data' criada com sucesso no banco de dados 'azurecost'.


In [9]:
from pyspark.sql import SparkSession
from delta.tables import DeltaTable
import os

DELTA_LAKE_PACKAGE = "io.delta:delta-core_2.12:3.3.2"

spark = SparkSession.builder \
    .appName("PySpark Delta Lake MinIO Save") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000") \
    .config("spark.hadoop.fs.s3a.access.key", os.getenv("KEY_ACCESS")) \
    .config("spark.hadoop.fs.s3a.secret.key", os.getenv("KEY_SECRETS")) \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider") \
    .getOrCreate()


# Caminho para a tabela Delta (no seu MinIO)
gold_path = "s3a://azurecost/gold"

# Inicializa objeto DeltaTable
delta_table = DeltaTable.forPath(spark, gold_path)

# Obtém todos os valores únicos da partição
partitions_df = delta_table.toDF().select("data_ref").distinct()

# Obtém o valor mais recente da partição
max_partition = partitions_df.agg({"data_ref": "max"}).collect()[0][0]
print(f"Última partição disponível: {max_partition}")

# Lê os dados somente da última partição
df = spark.read.format("delta").load(gold_path).filter(f"data_ref = '{max_partition}'")

df = df.select(
    "ResourceName",
    "SubscriptionId",
    "ResourceGroup",
    "Provider",
    "StatusRecourse",
    "PreTaxCost",
    "Pct_Change",
    "Currency",
    "UsageDate",
    "TendenciaCusto",
    "PrevisaoProxima"
)

df.show(truncate=False)

Última partição disponível: 2025-07-24
+----------------------+------------------------------------+-------------------------------------+-----------------+--------------+-------------------+----------+--------+-------------------+--------------+--------------------+
|ResourceName          |SubscriptionId                      |ResourceGroup                        |Provider         |StatusRecourse|PreTaxCost         |Pct_Change|Currency|UsageDate          |TendenciaCusto|PrevisaoProxima     |
+----------------------+------------------------------------+-------------------------------------+-----------------+--------------+-------------------+----------+--------+-------------------+--------------+--------------------+
|appfunckabum          |da483b95-1caf-404c-bfe4-36abef87f6e6|nintendoproject                      |microsoft.web    |Ativo         |0.0                |0.0       |BRL     |2025-07-24 17:05:00|Estável       |0.0                 |
|appfunckabum          |da483b95-1caf-404c-bf

In [10]:
jdbc_url = f"jdbc:postgresql://{DB_HOST}:{DB_PORT}/{DB_NAME}"

properties = {
    "user": DB_USER,
    "password": DB_PASSWORD,
    "driver": "org.postgresql.Driver"
}

df.write \
  .format("jdbc") \
  .option("url", jdbc_url) \
  .option("dbtable", "azure_cost_data") \
  .option("user", DB_USER) \
  .option("password", DB_PASSWORD) \
  .option("driver", "org.postgresql.Driver") \
  .mode("append") \
  .save()

print("Dados inseridos com sucesso na tabela 'azure_cost_data'.")  



Dados inseridos com sucesso na tabela 'azure_cost_data'.


In [ ]:
try:
    conn = psycopg2.connect(
        host=DB_HOST,
        port=DB_PORT,
        database=DB_NAME,
        user=DB_USER,
        password=DB_PASSWORD
    )
    conn.autocommit = True
    cur = conn.cursor()

    drop_table_query = "DROP TABLE IF EXISTS azure_cost_data;"
    cur.execute(drop_table_query)
    print(f"Tabela 'azure_cost_data' excluída com sucesso.")

    cur.close()

except psycopg2.Error as e:
    print(f"Erro ao excluir a tabela: {e}")

finally:
    if conn:
        conn.close()

In [ ]:
try:
    conn = psycopg2.connect(
        host=DB_HOST,
        port=DB_PORT,
        database="postgres",  # banco diferente do que será dropado
        user=DB_USER,
        password=DB_PASSWORD
    )
    conn.autocommit = True
    cur = conn.cursor()

    # Fecha conexões ativas no banco azurecost
    cur.execute("""
        SELECT pg_terminate_backend(pid) 
        FROM pg_stat_activity 
        WHERE datname = %s;
    """, (DB_NAME,))

    # Exclui o banco azurecost
    cur.execute(f"DROP DATABASE IF EXISTS {DB_NAME};")
    print(f"Banco de dados '{DB_NAME}' excluído com sucesso.")

    cur.close()

except psycopg2.Error as e:
    print(f"Erro ao excluir o banco de dados: {e}")

finally:
    if conn:
        conn.close()
